<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/matching/Matching_Algorithm_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CV Job Matching using Doc2Vec

### Coding
#### 1. Set up

In [1]:
# Install all dependencies
!pip install gensim
!pip install nltk
!pip install pandas
!pip install numpy
!pip install requests
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 14.8 MB/s eta 0:00:00
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 35.3s161.3 MiB [] 0% 28.4s161.3 MiB [] 0% 16.8s161.3 MiB [] 0% 10.4s161.3 MiB [] 1% 7.1s161.3 MiB [] 1% 8.0s161.3 MiB [] 1% 7.6s161.3 MiB [] 1% 7.0s161.3 MiB [] 2% 7.0s161.3 MiB [] 2% 6.8s161.3 MiB [] 2% 6.9s161.3 MiB [] 2% 7.0s161.3 MiB [] 3% 7.1s161.3 MiB [] 3% 7.4s161.3 MiB [] 3% 7.1s161.3 MiB [] 3% 6.6s161.3 MiB [] 4% 6.4s161.3 MiB [] 4% 6.6s161.3 MiB [] 4% 6.3s161.3 MiB [] 5% 6.0s161.3 MiB [] 5% 5.8s161.3 MiB [] 6% 5.7s161.3 MiB [] 6% 5.8s161.3 MiB [] 6% 6.2s161.3 MiB [] 7% 6.0s161.3 MiB [] 8% 5.8s161.3 MiB [] 8% 5.6s161.3 MiB [] 9% 5.4s161.3 MiB [] 9% 5.5s161.3 MiB [] 10% 5.5s161.3 MiB [] 10% 5.4s161.3 MiB [] 11% 5.5s161.3 MiB [] 11% 5.6s161.3 MiB [] 12% 5.6s161.3 MiB [] 12% 5.5s161.3 MiB [] 13% 5.3s161.3 MiB [] 14% 5.2s161.3 MiB [] 14% 5.3s161.3 MiB [] 15% 5.2s161.3 MiB [] 15% 5.1s161.3 MiB [] 16% 5.0s161.3 MiB [] 17% 4.9s161.3 MiB [] 18% 4.9s161.3 MiB [] 18% 4.8s16

In [2]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument,Word2Vec
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import re
import plotly.graph_objects as go
import nltk
nltk.download('all', download_dir='/usr/local/lib/nltk_data')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package avera

True

In [4]:
# Load data
jobs = pd.read_csv('/content/linkedin_tech_82k_git.csv')
profiles=pd.read_csv('/content/Lab_project.csv')

Keep only some columns to train the model

In [5]:
jobs =jobs[['Employment type', 'Industries', 'Seniority level',
       'company', 'description', 'education',
       'location', 'months_experience', 'post_url', 'salary', 'title']]

profiles=profiles[['about', 'certifications',
       'current_company',
       'education', 'experience','languages',
       'position', 'url', 'volunteer_experience', 'сourses']]


In [6]:
# Create a new column called 'data' and merge the values of the other columns into it
jobs['data'] = jobs[['Employment type', 'Industries', 'Seniority level',
       'company', 'description', 'education',
       'location', 'title']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
# Drop the individual columns if you no longer need them
jobs.drop(['Employment type', 'Industries', 'Seniority level',
       'company', 'description', 'education',
       'location', 'months_experience', 'post_url', 'salary', 'title'], axis=1, inplace=True)
# Preview the updated dataframe
print(jobs.head())

                                                data
0  Full-time Broadcast Media Mid-Senior level Cyb...
1  Full-time Hospital & Health Care, Medical Devi...
2  Full-time Computer Hardware, Computer Software...
3  Full-time Computer Hardware, Computer Software...
4  Full-time Computer Hardware, Computer Software...


In [7]:
# Create a new column called 'data' and merge the values of the other columns into it
profiles['data'] = profiles[['about', 'certifications',
       'current_company',
       'education', 'experience','languages',
       'position', 'сourses']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
# Drop the individual columns if you no longer need them
profiles.drop(['about', 'certifications',
       'current_company',
       'education', 'experience','languages',
       'position', 'url', 'volunteer_experience', 'сourses'], axis=1, inplace=True)
# Preview the updated dataframe
print(profiles.head())

                                                data
0  [] {"company_id":null,"industry":"Michigan Con...
1  [] {"company_id":"bcbsks","industry":"Blue Cro...
2  [] {"company_id":null,"industry":"Holm Corruga...
3  [{"meta":"Issued Sep 2021 See credential","sub...
4  [] {"company_id":"new-york-college-of-traditio...


#### 3. Tokenize data
We tokenize the words in the 'data' column and tag them with unique identifiers using the TaggedDocument class.

In [8]:
jobs_data = list(jobs['data'])
user_data=(list(profiles['data']))



In [9]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load NLTK stopwords
stop_words = set(stopwords.words('english'))

def preprocess_data(text):
    """
    Tokenize and clean text by removing stopwords, punctuation, and non-alphabetic tokens.
    """
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    filtered_tokens = [
        word for word in tokens
        if word.isalpha() and word not in stop_words  # Keep only alphabetic words and non-stopwords
    ]
    return filtered_tokens


In [ ]:
# # Preprocess and tag profiles
# tagged_profiles = [
#     TaggedDocument(words=preprocess_data(profile), tags=[f"profile_{i}"])
#     for i, profile in enumerate(user_data)
# ]

# # Preprocess and tag jobs
# tagged_jobs = [
#     TaggedDocument(words=preprocess_data(job), tags=[f"job_{i}"])
#     for i, job in enumerate(jobs_data)
# ]

# # Combine both lists
# tagged_data = tagged_profiles + tagged_jobs


KeyboardInterrupt: 

In [10]:
from gensim.models import Word2Vec

# Preprocess and tokenize profiles and jobs
tokenized_profiles = [preprocess_data(profile) for profile in user_data]
tokenized_jobs = [preprocess_data(job) for job in jobs_data]

# Combine all tokenized data
tokenized_data = tokenized_profiles + tokenized_jobs

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, min_count=2, epochs=10, workers=4)

# Get the vocabulary keys
keys = model.wv.key_to_index.keys()

# Print the length of the vocabulary keys
print(len(keys))


19993


In [ ]:
# model = Word2Vec(vector_size=100, min_count=2, epochs=10, workers=4)

In [ ]:
# # Vocabulary building
# model.build_vocab(tagged_data)
# # Get the vocabulary keys
# keys = model.wv.key_to_index.keys()
# # Print the length of the vocabulary keys
# print(len(keys))

#### 5. Train and save the model
Train the model on tagged data.

In [11]:
# Train the model
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
# Train for all desired epochs in a single call
    model.train(
    tokenized_data,
    total_examples=model.corpus_count,
    epochs=10,  # Set the total number of training epochs
    start_alpha=0.025,
    end_alpha=0.0001
)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/10


Training epoch 2/10


Training epoch 3/10


Training epoch 4/10


Training epoch 5/10


Training epoch 6/10


Training epoch 7/10


Training epoch 8/10


Training epoch 9/10


Training epoch 10/10
Model saved


#### 6. Inputs of CV and JD

In [91]:
profile_experiance= """Company Name: Next Career
Duration at company: 2 yrs 1 mo
Job Name: Headhunter (via Next Career)
Company: Flex
Job Type: Contract
Job Duration: Apr 2024 - May 2024 · 2 mos
Job Location: Israel · Hybrid
Additional Content: Recruitment as a Service
Skills: N/A
--------------------------------------------------
Job Name: Headhunter (via Next Career)
Company: Puzzle Projects LTD
Job Type: Contract
Job Duration: Feb 2024 - Apr 2024 · 3 mos
Job Location: Israel · Remote
Additional Content: Recruitment as a service for airspace and defence company.
Skills: N/A
--------------------------------------------------
Job Name: Talent Acquisition Manager (via Next Career)
Company: Discount Bank בנק דיסקונט
Job Type: Contract
Job Duration: 2023 - 2024 · 1 yr
Job Location: Israel · Hybrid
Additional Content: N/A
Skills: N/A
--------------------------------------------------"""

profile_education="""Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2019 - 2021
  Description: Master's degree, Organizational Behavior StudiesMaster's degree, Organizational Behavior Studies
  Skills:
  Additional Text: Grade: 94
Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2012 - 2014
  Description: Bachelor's degree, Human Resources Management and ServicesBachelor's degree, Human Resources Management and Services
  Skills:
  Additional Text: Grade: 95
Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2012 - 2014
  Description: Bachelor's degree, National Security Policy StudiesBachelor's degree, National Security Policy Studies
  Skills:
  Additional Text: """

profile_skills="""['Analytical Skills', 'Candidate Management', 'Co-sourcing',
   'Communication', 'Corporate Recruiting', 'Curriculum Vitae (CV)', 'English', 'Executive Search',
    'Game Theory', 'Hiring', 'Human Resources (HR)', 'Interpersonal Skills', 'Interview Preparation',
     'Leadership', 'LinkedIn', 'Microsoft Excel', 'Problem Solving', 'Recruiting',
      'Recruitment-to-Recruitment', 'Resume Writing', 'Sales', 'Sourcing', 'Start-up Ventures',
       'Statistical Data Analysis', 'Statistics', 'Strategic Sourcing', 'Strategy', 'Vacancies',
        'Venture Capital']"""

In [34]:
print(profile_skills)

Unique skills: ['Analytical Skills', 'Candidate Management', 'Co-sourcing',
   'Communication', 'Corporate Recruiting', 'Curriculum Vitae (CV)', 'English', 'Executive Search',
    'Game Theory', 'Hiring', 'Human Resources (HR)', 'Interpersonal Skills', 'Interview Preparation',
     'Leadership', 'LinkedIn', 'Microsoft Excel', 'Problem Solving', 'Recruiting',
      'Recruitment-to-Recruitment', 'Resume Writing', 'Sales', 'Sourcing', 'Start-up Ventures',
       'Statistical Data Analysis', 'Statistics', 'Strategic Sourcing', 'Strategy', 'Vacancies',
        'Venture Capital']


In [90]:
from playwright.async_api import async_playwright
import asyncio

async def scrape_job_posting(job_url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0")
        page = await context.new_page()

        await page.goto(job_url)
        await asyncio.sleep(5)  # Wait for 5 seconds

        # Saving the page's HTML content for debugging purposes
        html = await page.content()
        with open("page_debug.html", "w", encoding="utf-8") as f:
            f.write(html)
        print("Saved page content to page_debug.html")

        # Taking a screenshot of the page for verification
        await page.screenshot(path="screenshot.png", full_page=True)
        print("Screenshot saved as screenshot.png")


        # Extract job title
        job_title = await page.inner_text('h1.top-card-layout__title')  # Adjust selector
        company = await page.inner_text('a.topcard__org-name-link')  # Adjust selector
        location = await page.inner_text('span.topcard__flavor--bullet')  # Adjust selector
        description = await page.inner_text('div.show-more-less-html__markup')  # Adjust selector

        print(f"Job Title: {job_title}")
        print(f"Company: {company}")
        print(f"Location: {location}")
        print(f"Description: {description}")

        await browser.close()
        return job_title, company, location, description

job_url = "https://www.linkedin.com/jobs/view/4119256367"
job_title, company, location, description = await scrape_job_posting(job_url)

Saved page content to page_debug.html
Screenshot saved as screenshot.png
Job Title: Recruitment Lead
Company: Logistics company (NDA) 
Location:  Tel Aviv District, Israel
Description: HR Operations (Recruitment Lead)




We're seeking an HR Operations Manager in Tel Aviv.




You will be responsible for:

• Leading the recruitment team and overseeing mass recruitment for our business units

• Managing new-hire orientation and onboarding

Analyzing and optimizing business processes, advising on local law, and collaborating with providers

• Implementing an effective onboarding system

• Working with team metrics and KPIs

• Developing a high-quality recruitment strategy and structure




You might be a fit if you:

• Have solid relevant experience in talent development or recruitment

• Have extensive knowledge of local employment law

• Are a strong team player

• Are experienced in supporting various HR projects and drafting or amending policies based on business needs

• Are able to

- **Develop a function to pre-process input text**:

In [57]:
# parts = [profile_experiance, profile_education, profile_skills]
# profile_combined = " ".join(parts)

parts2 = [job_title, company, location, description]
jobs_combined = " ".join(parts2)



In [92]:
def preprocess_job_description(job_description):
    """
    Preprocess job description by removing excessive newlines and cleaning whitespace.
    Args:
        job_description (str): The raw job description text.
    Returns:
        str: The cleaned job description text.
    """
    # Replace multiple newlines with a single space
    cleaned_text = re.sub(r'\s*\n\s*', ' ', job_description)

    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    # Strip leading and trailing whitespace
    cleaned_text = cleaned_text.strip()

    return cleaned_text


In [93]:
import re

def preprocess_job_description(job_description):
    """
    Preprocess job description by removing excessive newlines and cleaning whitespace.
    """
    cleaned_text = re.sub(r'\s*\n\s*', ' ', job_description)  # Remove newlines
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Remove extra spaces
    return cleaned_text.strip()

def extract_skills_precise(job_description):
    """
    Extract skill-related sentences from a job description.
    """
    # Preprocess the job description first
    job_description = preprocess_job_description(job_description)

    # Split the description into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', job_description)

    # Keywords to identify skill-related sentences
    skill_keywords = [
        "hands-on experience",
        "familiarity with",
        "knowledge of",
        "experience with",
        "proficiency in",
        "solid understanding of",
        "ability to",
        "required skills",
        "technical skills",
        "skills and qualifications"
    ]

    # Filter sentences that contain any of the keywords
    skill_sentences = [
        sentence.strip() for sentence in sentences
        if any(keyword in sentence.lower() for keyword in skill_keywords)
    ]

    # Refine further by ensuring the sentence has meaningful content
    refined_sentences = [
        sentence for sentence in skill_sentences
        if len(sentence.split()) > 5  # Avoid very short or incomplete sentences
    ]

    return refined_sentences



# Extract precise skill-related sentences
job_skills_sentences = extract_skills_precise(description)

print("Extracted Skill Sentences:", job_skills_sentences)


Extracted Skill Sentences: ['You will be responsible for: • Leading the recruitment team and overseeing mass recruitment for our business units • Managing new-hire orientation and onboarding Analyzing and optimizing business processes, advising on local law, and collaborating with providers • Implementing an effective onboarding system • Working with team metrics and KPIs • Developing a high-quality recruitment strategy and structure You might be a fit if you: • Have solid relevant experience in talent development or recruitment • Have extensive knowledge of local employment law • Are a strong team player • Are experienced in supporting various HR projects and drafting or amending policies based on business needs • Are able to comfortably deal with ambiguity, interpret non-verbal communication, and understand cultural nuances • Are able to work independently and effectively in a fast-paced and changing environment • Possess excellent verbal and written communication skills • Are profic

In [94]:
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords

# Extended stopwords for job descriptions
stop_words = set(stopwords.words('english')).union({
    'have', 'has','you' 'solid', 'relevant', 'extensive', 'proven', 'experience', 'possess', 'ability', 'skills'
})

def extract_relevant_phrases(sentence):
    """
    Extract relevant phrases from a sentence by removing unnecessary words and focusing on key phrases.
    """
    # Remove unnecessary phrases using regex
    cleaned_sentence = re.sub(r'\b(have|has|possess|proven|solid|relevant|extensive)\b', '', sentence, flags=re.IGNORECASE)
    cleaned_sentence = re.sub(r'\b(experience (in|with|on|of|for))\b', '', cleaned_sentence, flags=re.IGNORECASE)

    # Tokenize and remove stopwords
    tokens = word_tokenize(cleaned_sentence.lower())
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]

    # Extract noun phrases using POS tagging
    pos_tags = pos_tag(tokens)
    relevant_phrases = [word for word, tag in pos_tags if tag.startswith('NN')]  # Keep only nouns

    return ' '.join(relevant_phrases)

def process_job_description(description):
    """
    Process job description to split into sentences and extract relevant parts.
    """
    # Split the description into sentences
    sentences = re.split(r'[\n•]', description)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Extract relevant phrases from each sentence
    processed_sentences = [extract_relevant_phrases(sentence) for sentence in sentences]

    # Remove empty results
    return [phrase for phrase in processed_sentences if phrase.strip()]


In [102]:
# Refined function to process job description
def process_job_description(description):
    """
    Process job description to split into sentences and extract relevant parts.
    """
    # Split the description into sentences
    sentences = re.split(r'[\n•]', description)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Extract relevant phrases from each sentence
    processed_sentences = [extract_relevant_phrases(sentence) for sentence in sentences]

    # Remove empty results
    return [phrase for phrase in processed_sentences if phrase.strip()]

# Refined phrase extraction function
def extract_relevant_phrases(sentence):
    """
    Extract relevant phrases from a sentence by removing unnecessary words and focusing on key phrases.
    """
    # Remove unnecessary phrases using regex
    cleaned_sentence = re.sub(r'\b(have|has|possess|proven|solid|relevant|extensive)\b', '', sentence, flags=re.IGNORECASE)
    cleaned_sentence = re.sub(r'\b(experience (in|with|on|of|for))\b', '', cleaned_sentence, flags=re.IGNORECASE)

    # Tokenize and remove stopwords
    tokens = word_tokenize(cleaned_sentence.lower())
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]

    # Extract noun phrases using POS tagging
    pos_tags = pos_tag(tokens)
    relevant_phrases = [word for word, tag in pos_tags if tag.startswith('NN')]  # Keep only nouns

    return ' '.join(relevant_phrases)

# Updated Workflow

# Process profile skills
profile_skills_list = eval(profile_skills)  # Convert string to list
profile_skills_processed = [preprocess(skill) for skill in profile_skills_list]

# Process job description using refined function
job_skills_processed = process_job_description(job_description)

# Encode the processed skills
profile_embeddings = model.encode(profile_skills_processed, convert_to_tensor=True)
job_embeddings = model.encode(job_skills_processed, convert_to_tensor=True)

# Compute cosine similarities
cosine_similarities = util.pytorch_cos_sim(job_embeddings, profile_embeddings)

# Adjust similarities based on threshold
similarity_threshold = 0.5
adjusted_similarities = []
total_similarity = 0  # Track the total similarity
comparison_count = 0  # Track the number of comparisons

for i, job_skill in enumerate(job_skills_processed):
    # Get all similarity scores for the current job skill
    similarity_row = cosine_similarities[i].tolist()

    # Check if any similarity exceeds the threshold
    max_similarity = max(similarity_row)
    if max_similarity > similarity_threshold:
        # If above threshold, set all similarities in the row to the maximum
        similarity_row = [1.0] * len(similarity_row)

    # Add the adjusted row to the results
    adjusted_similarities.append(similarity_row)

    # Update total similarity and comparison count
    total_similarity += sum(similarity_row)
    comparison_count += len(similarity_row)

# Calculate overall similarity as the average
overall_similarity = total_similarity / comparison_count

# Display adjusted results
print("Adjusted Similarities:")
for i, job_skill in enumerate(job_skills_processed):
    for j, profile_skill in enumerate(profile_skills_processed):
        similarity_score = adjusted_similarities[i][j]
        print(f"Similarity between job skill '{job_skill}' and profile skill '{profile_skill}': {similarity_score:.2f}")

# Display overall similarity
print(f"\nOverall Similarity: {overall_similarity:.2f}")


Adjusted Similarities:
Similarity between job skill 'hr operations lead' and profile skill 'analytical skills': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'candidate management': 1.00
Similarity between job skill 'hr operations lead' and profile skill '': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'communication': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'corporate recruiting': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'curriculum vitae cv': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'english': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'executive search': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'game theory': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'hiring': 1.00
Similarity between job skill 'hr operations lead' and profile skill 'human res

In [99]:
max_scores = []

for j, candidate_skill in enumerate(profile_skill):
    max_score = 0
    for i, job_skill in enumerate(job_skills_sentences):
        similarity_score = cosine_similarities[i][j].item()
        if similarity_score > max_score:
            max_score = similarity_score
    max_scores.append(max_score)

overall_score = np.mean(max_scores)

print(f"Overall similarity score for the candidate: {overall_score:.2f}")

Overall similarity score for the candidate: 0.26


In [32]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Ensure the input is a string (not tokenized yet)
    if isinstance(text, list):  # If it's already tokenized, return as-is
        return text
    # Lowercase and remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    # Tokenize
    words = text.split()
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words


In [16]:
# Apply to CV and JD
input_CV = preprocess_text(profile_combined)
input_JD = preprocess_text(jobs_combined)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# def get_tfidf_weighted_vector(model, document, tfidf_weights):
#     words = document.split()
#     word_vectors = [model.wv[word] * tfidf_weights.get(word, 0) for word in words if word in model.wv]
#     if not word_vectors:
#         return np.zeros(model.vector_size)
#     return np.mean(word_vectors, axis=0)

# # Example TF-IDF setup
# tfidf = TfidfVectorizer()
# tfidf.fit([input_CV, input_JD])  # Fit on both the CV and Job Description
# tfidf_weights = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))


AttributeError: 'list' object has no attribute 'lower'

In [17]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('cv_job_maching.model')  # Pretrained model for semantic similarity

def encode_document(model, document):
    """
    Encodes a document (string or list of sentences) into a single vector
    by averaging the sentence embeddings.
    """
    embeddings = model.encode(document)
    if len(embeddings.shape) == 2:  # If multiple sentence embeddings
        return np.mean(embeddings, axis=0)  # Average them
    return embeddings  # Return as is if already a single vector

# Encode the LinkedIn profile and job description
v1 = encode_document(model, input_CV)
v2 = encode_document(model, input_JD)

# Compute cosine similarity
similarity = cosine_similarity([v1], [v2])[0][0] * 100
print(f"Similarity: {round(similarity, 2)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: cv_job_maching.model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [21]:
# from transformers import AutoModel, AutoTokenizer
# import torch

# # Load SciBERT model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained('cv_job_maching.model')
# model = AutoModel.from_pretrained('cv_job_maching.model')

# def preprocess_input(text):
#     # Ensure input is a string
#     if isinstance(text, list):
#         text = ' '.join(text)  # Join list into string
#     # Convert to lowercase and apply additional preprocessing if needed
#     return text.lower()

# def encode_text(text):
#     # Preprocess the input text
#     text = preprocess_input(text)
#     # Tokenize and encode
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Return the mean of the last hidden state as the text embedding
#     return outputs.last_hidden_state.mean(dim=1).squeeze()



# # Process and encode inputs
# v1 = encode_text(input_CV)
# v2 = encode_text(input_JD)

# # Compute cosine similarity
# similarity = torch.nn.functional.cosine_similarity(v1, v2, dim=0)
# print(f"Similarity: {similarity.item() * 100:.2f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: cv_job_maching.model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

#### 7. Matching
Using the trained model, we infer the document vectors for the resume and job description. Then, we calculate the cosine similarity between the two vectors to determine the match between the resume and the job description.

In [ ]:
# # Model evaluation
# model = Doc2Vec.load('cv_job_maching.model')
# v1 = model.infer_vector(input_CV.split())
# v2 = model.infer_vector(input_JD.split())
# similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
# print(round(similarity, 2))

AttributeError: 'Word2Vec' object has no attribute 'infer_vector'

In [ ]:
# from numpy.linalg import norm
# import numpy as np

# # Load the trained Word2Vec model
# model = Word2Vec.load('cv_job_maching.model')

# def get_document_vector(model, document):
#     """
#     Compute the document vector by averaging word vectors.
#     Words not in the model's vocabulary are ignored.
#     """
#     words = document.split()
#     word_vectors = [model.wv[word] for word in words if word in model.wv]
#     if not word_vectors:  # Handle empty or unknown words
#         return np.zeros(model.vector_size)
#     return np.mean(word_vectors, axis=0)

# # Compute document vectors for the input CV and Job Description
# v1 = get_document_vector(model, input_CV)
# v2 = get_document_vector(model, input_JD)

# # Compute cosine similarity
# similarity = 100 * np.dot(v1, v2) / (norm(v1) * norm(v2))
# print(round(similarity, 2))


AttributeError: 'list' object has no attribute 'split'

#### 8. Visualization and Notification

In [23]:
# Visualization
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = similarity,
    mode = "gauge+number",
    title = {'text': "Matching percentage (%)"},
    #delta = {'reference': 100},
    gauge = {
        'axis': {'range': [0, 100]},
        'steps' : [
            {'range': [0, 50], 'color': "#FFB6C1"},
            {'range': [50, 70], 'color': "#FFFFE0"},
            {'range': [70, 100], 'color': "#90EE90"}
        ],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 100}}))

fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
fig.show()

# Print notification
if similarity < 50:
    print(colored("Low chance, need to modify your CV!", "red", attrs=["bold"]))
elif similarity >= 50 and similarity < 80:
    print(colored("Good chance but you can improve further!", "yellow", attrs=["bold"]))
else:
    print(colored("Excellent! You can submit your CV.", "green", attrs=["bold"]))

Excellent! You can submit your CV.
